In [1]:
import datetime
import numpy as np
import tensorflow as tf
import sys
from pandas import read_csv
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, GRU
from keras.models import model_from_json
import matplotlib.pyplot as plt
from IPython.display import clear_output

from IPython.display import SVG
np.random.seed(123)  # for reproducibility

# Extra part for memory management

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [2]:
def Make_LSTM_StepUp(train_index, val_index, test_index, state_condition='Stateless', shuffle='True', 
                     peak_hour=1, pred_hour=24, LSTM_hour=48, num_unit=240, num_epoch=1200, optimizer='Adam', loss_ftn='mean_squared_error'):
    print("StepUp Output!")
    
    len_PRED = pred_hour/peak_hour
    len_LSTM = LSTM_hour/peak_hour
    
    len_PRED = int(len_PRED)
    len_LSTM = int(len_LSTM)
    
    file_condition = str(peak_hour)+'hPEAK_'+str(pred_hour)+'hPrediction_'+str(LSTM_hour)+'hLSTM'
    
    if len_PRED <= 0:
        print("You entered wrong predction hour or peak hour")
        sys.eixt()
    if len_LSTM <= 0:
        print("You entered wrong LSTM hour or peak hour")
        sys.eixt()
       
    allDataSet = read_csv('C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SEP\\GOES_LSTM_csv\\GOES15_pgt10_'+str(peak_hour)+'hPEAK_1hPrediction_'+str(LSTM_hour)+'hLSTM_log.csv', sep=',')
    allDataSet_size = len(allDataSet)
    
    print("allDataSet: ", allDataSet.shape)
    print("allDataSet_size: ", allDataSet_size)
    print("allDataSet Type: ", type(allDataSet))
    print("allDataSet_size Type: ", type(allDataSet_size))
    
    data_count = np.arange(len_LSTM)+1
    data_count = data_count* (-1)
    data_count = data_count +len_LSTM+1
    data_title = []
    for bi in range(len(data_count)):
        data_title.append('b'+str(int(data_count[bi])))
        
    pred_count = np.arange(len_PRED)
    pred_title = []
    for ai in range(len(pred_count)):
        pred_title.append('a'+str(int(pred_count[ai])))

    #flux = allDataSet[['Year','Mon','Day','Hour','Min','b1','b2','b3','b4','b5','b6']].values
    flux = allDataSet[data_title].values
#    pred_flux = allDataSet[pred_title].values
    now_flux = allDataSet['a0'].values
    now_Year = allDataSet['Year'].values
    now_Mon = allDataSet['Mon'].values
    now_Day = allDataSet['Day'].values
    now_Hour = allDataSet['Hour'].values
    now_Min = allDataSet['Min'].values
     
    d_flux = np.array([flux])
#    d_pred_flux = np.array([pred_flux])
    d_now_flux = np.array([now_flux])
    d_now_Year = np.array([now_Year])
    d_now_Mon = np.array([now_Mon])
    d_now_Day = np.array([now_Day])
    d_now_Hour = np.array([now_Hour])
    d_now_Min = np.array([now_Min])
    
    
    #d_flux = np.array(d_flux).reshape(allDataSet_size, 1, 6)
    #inputFluxShape = (1,6)    
    d_flux = np.array(d_flux).reshape(allDataSet_size, len_LSTM,1) ### d_flux = np.array(d_flux).reshape(allDataSet_size, 1, len_LSTM)
#    d_pred_flux = np.array(d_pred_flux).reshape(allDataSet_size, len_PRED)
    d_now_flux = np.array([d_now_flux]).reshape(allDataSet_size, 1)
    d_now_Year = np.array([d_now_Year]).reshape(allDataSet_size, 1)
    d_now_Mon = np.array([d_now_Mon]).reshape(allDataSet_size, 1)
    d_now_Day = np.array([d_now_Day]).reshape(allDataSet_size, 1)
    d_now_Hour = np.array([d_now_Hour]).reshape(allDataSet_size, 1)
    d_now_Min = np.array([d_now_Min]).reshape(allDataSet_size, 1)
    

    def M_LSTM_Stateful(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,batch_input_shape=shapes, stateful=True, activation='relu'))
        model.add(Dense(1))
        model.summary()
        return model
            
    def M_LSTM_Stateless(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,input_shape=shapes, stateful=False, activation='relu'))  # model.add(LSTM(num_unit,input_shape=shapes, return_sequences=True, stateful=False))
#        model.add(Dropout(0.3))
#        model.add(LSTM(num_unit,input_shape=shapes, return_sequences=True, stateful=False))
#        model.add(Dropout(0.3))
#        model.add(LSTM(num_unit,input_shape=shapes, stateful=False))
        model.add(Dense(1))
        model.summary()
        return model


    class PlotLosses(keras.callbacks.Callback):
        def on_train_begin(self, logs={}):
            self.i = 0
            self.x = []
            self.losses = []
            self.val_losses = []

            self.fig = plt.figure()

            self.logs = []

        def on_epoch_end(self, epoch, logs={}):

            self.logs.append(logs)
            self.x.append(self.i)
            self.losses.append(logs.get('loss'))
            self.val_losses.append(logs.get('val_loss'))
            self.i += 1
            clear_output(wait=True)
            plt.plot(self.x, self.losses, label="loss")
            plt.plot(self.x, self.val_losses, label="val_loss")
            plt.legend()
            plt.show();
            

    if shuffle is 'True':
        weightfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SEP\\GOES_LSTM_TEST_WEIGHT\\Layer1_shuffled_'+state_condition+'_StepUpOutput_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_WEIGHT'
        testfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SEP\\GOES_LSTM_TEST_RESULT\\Layer1_shuffled_'+state_condition+'_StepUpOutput_TEST_'+file_condition+'_Proton_LSTM.csv'
    else:
        if shuffle is 'False':
            weightfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SEP\\GOES_LSTM_TEST_WEIGHT\\Layer1_'+state_condition+'_StepUpOutput_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_WEIGHT'
            testfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SEP\\GOES_LSTM_TEST_RESULT\\Layer1_'+state_condition+'_StepUpOutput_TEST_'+file_condition+'_Proton_LSTM.csv'
            

    plot_losses = PlotLosses()
    
    if state_condition is 'Stateful':
        inputFluxShape = (1,len_LSTM, 1)
        model = M_LSTM_Stateful(inputFluxShape)

    else:
        if state_condition is 'Stateless':
            inputFluxShape = (len_LSTM,1)  #####  inputFluxShape = (1, len_LSTM)
            model = M_LSTM_Stateless(inputFluxShape)
        else:
            print("You enterd wrong state_condition!")
            sys.exit()
 

    val_index  = train_index + val_index
    test_index = val_index + test_index
    
    trainX = d_flux[:train_index]
    trainY = d_now_flux[:train_index]
    trainYear = d_now_Year[:train_index]
    trainMon = d_now_Mon[:train_index]
    trainDay = d_now_Day[:train_index]
    trainHour = d_now_Hour[:train_index]
    trainMin = d_now_Min[:train_index]
    
    valX = d_flux[train_index:val_index]
    valY = d_now_flux[train_index:val_index]
    valYear = d_now_Year[train_index:val_index]
    valMon = d_now_Mon[train_index:val_index]
    valDay = d_now_Day[train_index:val_index]
    valHour = d_now_Hour[train_index:val_index]
    valMin = d_now_Min[train_index:val_index]
    
    testX = d_flux[val_index:test_index]
    testY = d_now_flux[val_index:test_index]
    testYear = d_now_Year[val_index:test_index]
    testMon = d_now_Mon[val_index:test_index]
    testDay = d_now_Day[val_index:test_index]
    testHour = d_now_Hour[val_index:test_index]
    testMin = d_now_Min[val_index:test_index]
    
    model.compile(loss=loss_ftn, optimizer=optimizer, metrics=['acc'])
    
    
    if state_condition is 'Stateful':    
        for epoch_idx in range(num_epoch):
            if shuffle is 'True':
                model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2, shuffle=True, validation_data=(valX, valY))
                model.reset_states()
            else:
                if shuffle is 'False':
                    model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2, shuffle=False, validation_data=(valX, valY))
                    model.reset_states()
            print(epoch_idx)
    else:
        if state_condition is 'Stateless': 
            if shuffle is 'True':
                model.fit(trainX, trainY, epochs=num_epoch, batch_size=1400, verbose=2, shuffle=True, validation_data=(valX, valY))
            else:
                if shuffle is 'False':
                    model.fit(trainX, trainY, epochs=num_epoch, batch_size=1400, verbose=2, shuffle=False, validation_data=(valX, valY))

        
    model_json = model.to_json()
    with open(weightfileName + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(weightfileName + ".h5")
    print("Saved model to disk")

    json_file = open(weightfileName + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    loaded_model.load_weights(weightfileName + ".h5")
    print("Loaded model from disk")
    loaded_model.compile(loss=loss_ftn, optimizer=optimizer, metrics=['acc'])
    

    #date_testY = allDataSet['date'].values[val_index:test_index]
    f = open(testfileName, 'w')
    f.write('StepUp Output_'+str(num_epoch)+'epochs_'+str(num_unit)+'units_'+optimizer + ' optimizer_' + loss_ftn+' loss function')
    f.write('\n')
    f.write('Year' + ','+ 'Month' + ','+ 'Day' + ','+ 'Hour' + ','+ 'Min' + ',' + 'Ground')
    for i in range(len_PRED):
        f.write(','+ 'a'+str(i))
    f.write('\n')
    
        
    for idx2 in range(0, len(testX)):
        
        
        NowY = testY[idx2:idx2+1]

        Year = testYear[idx2][0]
        Mon = testMon[idx2][0]
        Day = testDay[idx2][0]
        Hour = testHour[idx2][0]
        Min = testMin[idx2][0]

        if idx2 != len(testX)-1:
            Year2 = testYear[idx2+1][0]
            Mon2 = testMon[idx2+1][0]
            Day2 = testDay[idx2+1][0]
            Hour2 = testHour[idx2+1][0]
            Min2 = testMin[idx2+1][0]

            pre_time = datetime.datetime(Year, Mon, Day, Hour, Min)
            next_time = datetime.datetime(Year2, Mon2, Day2, Hour2, Min2)
            Second_Term = peak_hour*3600   

            Diff = next_time - pre_time
            Diff = Diff.total_seconds()
            Diff = Diff/Second_Term
            Diff = Diff-1
        
        pred = []
        pred_in = testX[idx2:idx2+1]  #    (1, 1, 96)            

        for pred_n in range(len_PRED):
            resultY = model.predict(pred_in)
            pred.append(resultY)

            pred_in = np.append(pred_in,resultY)
            pred_in = pred_in[1:]
            pred_in = pred_in.reshape(1, len_LSTM, 1)

        date = str(Year) + ',' + str(Mon) + ',' + str(Day) + ',' + str(Hour) + ',' + str(Min)
        f.write(date)
        f.write(',' + str(NowY[0][0]))

        for nwr in range(len_PRED):
            f.write(',' + str(pred[nwr][0][0]))   

        f.write('\n')
        pred_in = []
        resultY = []
        print(idx2)
        
        if idx2 != len(testX)-1:
            if Diff >= 1:                            
                Term = datetime.timedelta(seconds=Second_Term)
                ddd = pre_time
                for n_diff in range(int(Diff)):    
                    ddd = ddd + Term
                    #print(str(ddd.year) + ',' + str(ddd.month) + ',' + str(ddd.day) + ',' + str(ddd.hour) + ',' + str(ddd.minute) + '\n')
                    f.write(str(ddd.year) + ',' + str(ddd.month) + ',' + str(ddd.day) + ',' + str(ddd.hour) + ',' + str(ddd.minute))
                    for wr in range(len_PRED+1):
                        f.write(',' + str(-99999))
                    f.write('\n')

    f.close()

###################################################################################################

In [3]:
def Make_LSTM_Onetime(train_index, val_index, test_index, state_condition='Stateless', shuffle='True', 
                      peak_hour=1, pred_hour=24, LSTM_hour=48, num_unit=240, num_epoch=1200, optimizer='Adam', loss_ftn='mean_squared_error'): 
    print("Onetime Output!")

    len_PRED = pred_hour/peak_hour
    len_LSTM = LSTM_hour/peak_hour

    len_PRED = int(len_PRED)
    len_LSTM = int(len_LSTM)

    file_condition = str(peak_hour)+'hPEAK_'+str(pred_hour)+'hPrediction_'+str(LSTM_hour)+'hLSTM'

    if len_PRED <= 0:
        print("You entered wrong predction hour or peak hour")
        sys.eixt()
    if len_LSTM <= 0:
        print("You entered wrong LSTM hour or peak hour")
        sys.eixt()

    allDataSet = read_csv('C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SEP\\GOES_LSTM_csv\\GOES15_pgt10_'+file_condition+'_log.csv', sep=',')
    allDataSet_size = len(allDataSet)

    print("allDataSet: ", allDataSet.shape)
    print("allDataSet_size: ", allDataSet_size)
    print("allDataSet Type: ", type(allDataSet))
    print("allDataSet_size Type: ", type(allDataSet_size))

    data_count = np.arange(len_LSTM)+1
    data_count = data_count* (-1)
    data_count = data_count +len_LSTM+1
    data_title = []
    for bi in range(len(data_count)):
        data_title.append('b'+str(int(data_count[bi])))

    pred_count = np.arange(len_PRED)
    pred_title = []
    for ai in range(len(pred_count)):
        pred_title.append('a'+str(int(pred_count[ai])))

    #flux = allDataSet[['Year','Mon','Day','Hour','Min','b1','b2','b3','b4','b5','b6']].values
    flux = allDataSet[data_title].values
    pred_flux = allDataSet[pred_title].values
    now_flux = allDataSet['a0'].values
    now_Year = allDataSet['Year'].values
    now_Mon = allDataSet['Mon'].values
    now_Day = allDataSet['Day'].values
    now_Hour = allDataSet['Hour'].values
    now_Min = allDataSet['Min'].values

    d_flux = np.array([flux])
    d_pred_flux = np.array([pred_flux])
    d_now_flux = np.array([now_flux])
    d_now_Year = np.array([now_Year])
    d_now_Mon = np.array([now_Mon])
    d_now_Day = np.array([now_Day])
    d_now_Hour = np.array([now_Hour])
    d_now_Min = np.array([now_Min])


    #d_flux = np.array(d_flux).reshape(allDataSet_size, 1, 6)
    #inputFluxShape = (1,6)    
    d_flux = np.array(d_flux).reshape(allDataSet_size, len_LSTM,1) #### d_flux = np.array(d_flux).reshape(allDataSet_size, 1, len_LSTM)
    d_pred_flux = np.array(d_pred_flux).reshape(allDataSet_size, len_PRED)
    d_now_flux = np.array([d_now_flux]).reshape(allDataSet_size, 1)
    d_now_Year = np.array([d_now_Year]).reshape(allDataSet_size, 1)
    d_now_Mon = np.array([d_now_Mon]).reshape(allDataSet_size, 1)
    d_now_Day = np.array([d_now_Day]).reshape(allDataSet_size, 1)
    d_now_Hour = np.array([d_now_Hour]).reshape(allDataSet_size, 1)
    d_now_Min = np.array([d_now_Min]).reshape(allDataSet_size, 1)


    def M_LSTM_Stateful(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,batch_input_shape=shapes, stateful=True, activation='relu'))
        model.add(Dense(len_PRED))
        model.summary()
        return model

    def M_LSTM_Stateless(shapes):
        model = Sequential()
        model.add(GRU(num_unit,input_shape=shapes, stateful=False, activation='relu'))   # model.add(LSTM(num_unit,input_shape=shapes, return_sequences = True, stateful=False))  
#        model.add(Dropout(0.3))
#        model.add(LSTM(num_unit,input_shape=shapes, return_sequences = True, stateful=False))
#        model.add(Dropout(0.3))
#        model.add(LSTM(num_unit,input_shape=shapes, stateful=False))
        model.add(Dense(len_PRED))
        model.summary()
        return model

    class PlotLosses(keras.callbacks.Callback):
        def on_train_begin(self, logs={}):
            self.i = 0
            self.x = []
            self.losses = []
            self.val_losses = []

            self.fig = plt.figure()

            self.logs = []

        def on_epoch_end(self, epoch, logs={}):

            self.logs.append(logs)
            self.x.append(self.i)
            self.losses.append(logs.get('loss'))
            self.val_losses.append(logs.get('val_loss'))
            self.i += 1
            clear_output(wait=True)
            plt.plot(self.x, self.losses, label="loss")
            plt.plot(self.x, self.val_losses, label="val_loss")
            plt.legend()
            plt.show();


    if shuffle is 'True':
        weightfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study\\SEP\\GOES_LSTM_TEST_WEIGHT\\Layer1_shuffled_'+state_condition+'_OnetimeOutput_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_WEIGHT'
        testfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study\\SEP\\GOES_LSTM_TEST_RESULT\\Layer1_shuffled_'+state_condition+'_OnetimeOutput_TEST_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units'+'_Proton_LSTM.csv'
    else:
        if shuffle is 'False':
            weightfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study\\SEP\\GOES_LSTM_TEST_WEIGHT\\Layer1_'+state_condition+'_OnetimeOutput_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_WEIGHT'
            testfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study\\SEP\\GOES_LSTM_TEST_RESULT\\Layer1_'+state_condition+'_OnetimeOutput_TEST_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units'+'_Proton_LSTM.csv'

    plot_losses = PlotLosses()

    if state_condition is 'Stateful':
        inputFluxShape = (1,len_LSTM, 1)
        model = M_LSTM_Stateful(inputFluxShape)

    else:
        if state_condition is 'Stateless':
            inputFluxShape = (len_LSTM,1)  #####  inputFluxShape = (1, len_LSTM)
            model = M_LSTM_Stateless(inputFluxShape)
        else:
            print("You enterd wrong state_condition!")
            sys.exit()

    
    SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))
    
    sys.exit()
    

    val_index  = train_index + val_index
    test_index = val_index + test_index

    trainX = d_flux[:train_index]
    trainY = d_pred_flux[:train_index]
    trainYear = d_now_Year[:train_index]
    trainMon = d_now_Mon[:train_index]
    trainDay = d_now_Day[:train_index]
    trainHour = d_now_Hour[:train_index]
    trainMin = d_now_Min[:train_index]

    valX = d_flux[train_index:val_index]
    valY = d_pred_flux[train_index:val_index]
    valYear = d_now_Year[train_index:val_index]
    valMon = d_now_Mon[train_index:val_index]
    valDay = d_now_Day[train_index:val_index]
    valHour = d_now_Hour[train_index:val_index]
    valMin = d_now_Min[train_index:val_index]

    testX = d_flux[val_index:test_index]
    testY = d_pred_flux[val_index:test_index]
    testNowY = d_now_flux[val_index:test_index]
    testYear = d_now_Year[val_index:test_index]
    testMon = d_now_Mon[val_index:test_index]
    testDay = d_now_Day[val_index:test_index]
    testHour = d_now_Hour[val_index:test_index]
    testMin = d_now_Min[val_index:test_index]

    model.compile(loss=loss_ftn, optimizer=optimizer, metrics=['acc'])


    if state_condition is 'Stateful':    
        for epoch_idx in range(num_epoch):
            if shuffle is 'True':
                model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2, shuffle=True, validation_data=(valX, valY))
                model.reset_states()
            else:
                if shuffle is 'False':
                    model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2, shuffle=False, validation_data=(valX, valY))
                    model.reset_states()
            print(epoch_idx)
    else:
        if state_condition is 'Stateless': 
            if shuffle is 'True':
                model.fit(trainX, trainY, epochs=num_epoch, batch_size=1400, verbose=2, shuffle=True, validation_data=(valX, valY))
            else:
                if shuffle is 'False':
                    model.fit(trainX, trainY, epochs=num_epoch, batch_size=1400, verbose=2, shuffle=False, validation_data=(valX, valY))


    model_json = model.to_json()
    with open(weightfileName + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(weightfileName + ".h5")
    print("Saved model to disk")

    json_file = open(weightfileName + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    loaded_model.load_weights(weightfileName + ".h5")
    print("Loaded model from disk")
    loaded_model.compile(loss=loss_ftn, optimizer=optimizer, metrics=['acc'])


    #date_testY = allDataSet['date'].values[val_index:test_index]
    f = open(testfileName, 'w')
    f.write('Onetime Output_'+str(num_epoch)+'epochs_'+str(num_unit)+'units_'+optimizer + ' optimizer_' + loss_ftn+' loss function')
    f.write('\n')
    f.write('Year' + ','+ 'Month' + ','+ 'Day' + ','+ 'Hour' + ','+ 'Min' + ',' + 'Ground')
    for i in range(len_PRED):
        f.write(','+ 'a'+str(i))
    f.write('\n')

    for idx2 in range(0, len(testX)):

        NowY = testNowY[idx2:idx2+1]

        Year = testYear[idx2][0]
        Mon = testMon[idx2][0]
        Day = testDay[idx2][0]
        Hour = testHour[idx2][0]
        Min = testMin[idx2][0]

        if idx2 != len(testX)-1:
            Year2 = testYear[idx2+1][0]
            Mon2 = testMon[idx2+1][0]
            Day2 = testDay[idx2+1][0]
            Hour2 = testHour[idx2+1][0]
            Min2 = testMin[idx2+1][0]

            pre_time = datetime.datetime(Year, Mon, Day, Hour, Min)
            next_time = datetime.datetime(Year2, Mon2, Day2, Hour2, Min2)
            Second_Term = peak_hour*3600   

            Diff = next_time - pre_time
            Diff = Diff.total_seconds()
            Diff = Diff/Second_Term
            Diff = Diff-1

        pred = []
        pred_in = testX[idx2:idx2+1]      

        resultY = model.predict(pred_in)

        date = str(Year) + ',' + str(Mon) + ',' + str(Day) + ',' + str(Hour) + ',' + str(Min)

        f.write(date)
        f.write(',' + str(NowY[0][0]))

        for nwr in range(len_PRED):
            f.write(',' + str(resultY[0][nwr]))   

        f.write('\n')
        pred_in = []
        resultY = []
        print(idx2)

        if idx2 != len(testX)-1:
            if Diff >= 1:                            
                Term = datetime.timedelta(seconds=Second_Term)
                ddd = pre_time
                for n_diff in range(int(Diff)):    
                    ddd = ddd + Term
                    #print(str(ddd.year) + ',' + str(ddd.month) + ',' + str(ddd.day) + ',' + str(ddd.hour) + ',' + str(ddd.minute) + '\n')
                    f.write(str(ddd.year) + ',' + str(ddd.month) + ',' + str(ddd.day) + ',' + str(ddd.hour) + ',' + str(ddd.minute))
                    for wr in range(len_PRED+1):
                        f.write(',' + str(-99999))
                    f.write('\n')

    f.close()
###################################################################################################

In [4]:
def Make_LSTM_Onetime_load(train_index, val_index, test_index, state_condition='Stateless', shuffle='True', 
                      peak_hour=1, pred_hour=24, LSTM_hour=48, num_unit=240, num_epoch=1200, optimizer='Adam', loss_ftn='mean_squared_error'): 
    print("Onetime Output!")

    len_PRED = pred_hour/peak_hour
    len_LSTM = LSTM_hour/peak_hour

    len_PRED = int(len_PRED)
    len_LSTM = int(len_LSTM)

    file_condition = str(peak_hour)+'hPEAK_'+str(pred_hour)+'hPrediction_'+str(LSTM_hour)+'hLSTM'

    if len_PRED <= 0:
        print("You entered wrong predction hour or peak hour")
        sys.eixt()
    if len_LSTM <= 0:
        print("You entered wrong LSTM hour or peak hour")
        sys.eixt()

    allDataSet = read_csv('C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SEP\\GOES_LSTM_csv\\GOES15_pgt10_'+file_condition+'_log.csv', sep=',')
    allDataSet_size = len(allDataSet)

    print("allDataSet: ", allDataSet.shape)
    print("allDataSet_size: ", allDataSet_size)
    print("allDataSet Type: ", type(allDataSet))
    print("allDataSet_size Type: ", type(allDataSet_size))

    data_count = np.arange(len_LSTM)+1
    data_count = data_count* (-1)
    data_count = data_count +len_LSTM+1
    data_title = []
    for bi in range(len(data_count)):
        data_title.append('b'+str(int(data_count[bi])))

    pred_count = np.arange(len_PRED)
    pred_title = []
    for ai in range(len(pred_count)):
        pred_title.append('a'+str(int(pred_count[ai])))

    #flux = allDataSet[['Year','Mon','Day','Hour','Min','b1','b2','b3','b4','b5','b6']].values
    flux = allDataSet[data_title].values
    pred_flux = allDataSet[pred_title].values
    now_flux = allDataSet['a0'].values
    now_Year = allDataSet['Year'].values
    now_Mon = allDataSet['Mon'].values
    now_Day = allDataSet['Day'].values
    now_Hour = allDataSet['Hour'].values
    now_Min = allDataSet['Min'].values

    d_flux = np.array([flux])
    d_pred_flux = np.array([pred_flux])
    d_now_flux = np.array([now_flux])
    d_now_Year = np.array([now_Year])
    d_now_Mon = np.array([now_Mon])
    d_now_Day = np.array([now_Day])
    d_now_Hour = np.array([now_Hour])
    d_now_Min = np.array([now_Min])


    #d_flux = np.array(d_flux).reshape(allDataSet_size, 1, 6)
    #inputFluxShape = (1,6)    
    d_flux = np.array(d_flux).reshape(allDataSet_size, len_LSTM,1) #### d_flux = np.array(d_flux).reshape(allDataSet_size, 1, len_LSTM)
    d_pred_flux = np.array(d_pred_flux).reshape(allDataSet_size, len_PRED)
    d_now_flux = np.array([d_now_flux]).reshape(allDataSet_size, 1)
    d_now_Year = np.array([d_now_Year]).reshape(allDataSet_size, 1)
    d_now_Mon = np.array([d_now_Mon]).reshape(allDataSet_size, 1)
    d_now_Day = np.array([d_now_Day]).reshape(allDataSet_size, 1)
    d_now_Hour = np.array([d_now_Hour]).reshape(allDataSet_size, 1)
    d_now_Min = np.array([d_now_Min]).reshape(allDataSet_size, 1)


    def M_LSTM_Stateful(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,batch_input_shape=shapes, stateful=True, activation='relu'))
        model.add(Dense(len_PRED))
        model.summary()
        return model

    def M_LSTM_Stateless(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,input_shape=shapes, stateful=False, activation='relu'))   # model.add(LSTM(num_unit,input_shape=shapes, return_sequences = True, stateful=False))  
#        model.add(Dropout(0.3))
#        model.add(LSTM(num_unit,input_shape=shapes, return_sequences = True, stateful=False))
#        model.add(Dropout(0.3))
#        model.add(LSTM(num_unit,input_shape=shapes, stateful=False))
        model.add(Dense(len_PRED))
        model.summary()
        return model

    class PlotLosses(keras.callbacks.Callback):
        def on_train_begin(self, logs={}):
            self.i = 0
            self.x = []
            self.losses = []
            self.val_losses = []

            self.fig = plt.figure()

            self.logs = []

        def on_epoch_end(self, epoch, logs={}):

            self.logs.append(logs)
            self.x.append(self.i)
            self.losses.append(logs.get('loss'))
            self.val_losses.append(logs.get('val_loss'))
            self.i += 1
            clear_output(wait=True)
            plt.plot(self.x, self.losses, label="loss")
            plt.plot(self.x, self.val_losses, label="val_loss")
            plt.legend()
            plt.show();


    if shuffle is 'True':
        weightfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SEP\\GOES_LSTM_TEST_WEIGHT\\Layer1_shuffled_'+state_condition+'_OnetimeOutput_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_WEIGHT'
        testfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SEP\\GOES_LSTM_TEST_RESULT\\Layer1_shuffled_'+state_condition+'_OnetimeOutput_TEST_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units'+'_Proton_LSTM.csv'
    else:
        if shuffle is 'False':
            weightfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SEP\\GOES_LSTM_TEST_WEIGHT\\Layer1_'+state_condition+'_OnetimeOutput_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_WEIGHT'
            testfileName = 'C:\\Users\\YKW\\GoogleDrive\\Study USB BackUp\\SEP\\GOES_LSTM_TEST_RESULT\\Layer1_'+state_condition+'_OnetimeOutput_TEST_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units'+'_Proton_LSTM.csv'

    plot_losses = PlotLosses()

    if state_condition is 'Stateful':
        inputFluxShape = (1,len_LSTM, 1)
        model = M_LSTM_Stateful(inputFluxShape)

    else:
        if state_condition is 'Stateless':
            inputFluxShape = (len_LSTM,1)  #####  inputFluxShape = (1, len_LSTM)
            model = M_LSTM_Stateless(inputFluxShape)
        else:
            print("You enterd wrong state_condition!")
            sys.exit()


    val_index  = train_index + val_index
    test_index = val_index + test_index

    trainX = d_flux[:train_index]
    trainY = d_pred_flux[:train_index]
    trainYear = d_now_Year[:train_index]
    trainMon = d_now_Mon[:train_index]
    trainDay = d_now_Day[:train_index]
    trainHour = d_now_Hour[:train_index]
    trainMin = d_now_Min[:train_index]

    valX = d_flux[train_index:val_index]
    valY = d_pred_flux[train_index:val_index]
    valYear = d_now_Year[train_index:val_index]
    valMon = d_now_Mon[train_index:val_index]
    valDay = d_now_Day[train_index:val_index]
    valHour = d_now_Hour[train_index:val_index]
    valMin = d_now_Min[train_index:val_index]

    testX = d_flux[val_index:test_index]
    testY = d_pred_flux[val_index:test_index]
    testNowY = d_now_flux[val_index:test_index]
    testYear = d_now_Year[val_index:test_index]
    testMon = d_now_Mon[val_index:test_index]
    testDay = d_now_Day[val_index:test_index]
    testHour = d_now_Hour[val_index:test_index]
    testMin = d_now_Min[val_index:test_index]

    model.compile(loss=loss_ftn, optimizer=optimizer, metrics=['acc'])


#    if state_condition is 'Stateful':    
#        for epoch_idx in range(num_epoch):
#            if shuffle is 'True':
#                model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2, shuffle=True, validation_data=(valX, valY))
#                model.reset_states()
#            else:
#                if shuffle is 'False':
#                    model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2, shuffle=False, validation_data=(valX, valY))
#                    model.reset_states()
#            print(epoch_idx)
#    else:
#        if state_condition is 'Stateless': 
#            if shuffle is 'True':
#                model.fit(trainX, trainY, epochs=num_epoch, batch_size=1400, verbose=2, shuffle=True, validation_data=(valX, valY))
#            else:
#                if shuffle is 'False':
#                    model.fit(trainX, trainY, epochs=num_epoch, batch_size=1400, verbose=2, shuffle=False, validation_data=(valX, valY))
#    model_json = model.to_json()
#    with open(weightfileName + ".json", "w") as json_file:
#        json_file.write(model_json)
#    model.save_weights(weightfileName + ".h5")
#    print("Saved model to disk")

    json_file = open(weightfileName + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    loaded_model.load_weights(weightfileName + ".h5")
    print("Loaded model from disk")
    loaded_model.compile(loss=loss_ftn, optimizer=optimizer, metrics=['acc'])


    #date_testY = allDataSet['date'].values[val_index:test_index]
    f = open(testfileName, 'w')
    f.write('Onetime Output_'+str(num_epoch)+'epochs_'+str(num_unit)+'units_'+optimizer + ' optimizer_' + loss_ftn+' loss function')
    f.write('\n')
    f.write('Year' + ','+ 'Month' + ','+ 'Day' + ','+ 'Hour' + ','+ 'Min' + ',' + 'Ground')
    for i in range(len_PRED):
        f.write(','+ 'a'+str(i))
    f.write('\n')

    for idx2 in range(0, len(testX)):

        NowY = testNowY[idx2:idx2+1]

        Year = testYear[idx2][0]
        Mon = testMon[idx2][0]
        Day = testDay[idx2][0]
        Hour = testHour[idx2][0]
        Min = testMin[idx2][0]

        if idx2 != len(testX)-1:
            Year2 = testYear[idx2+1][0]
            Mon2 = testMon[idx2+1][0]
            Day2 = testDay[idx2+1][0]
            Hour2 = testHour[idx2+1][0]
            Min2 = testMin[idx2+1][0]

            pre_time = datetime.datetime(Year, Mon, Day, Hour, Min)
            next_time = datetime.datetime(Year2, Mon2, Day2, Hour2, Min2)
            Second_Term = peak_hour*3600   

            Diff = next_time - pre_time
            Diff = Diff.total_seconds()
            Diff = Diff/Second_Term
            Diff = Diff-1

        pred = []
        pred_in = testX[idx2:idx2+1]      

        resultY = loaded_model.predict(pred_in)

        date = str(Year) + ',' + str(Mon) + ',' + str(Day) + ',' + str(Hour) + ',' + str(Min)

        f.write(date)
        f.write(',' + str(NowY[0][0]))

        for nwr in range(len_PRED):
            f.write(',' + str(resultY[0][nwr]))   

        f.write('\n')
        pred_in = []
        resultY = []
        print(idx2)

        if idx2 != len(testX)-1:
            if Diff >= 1:                            
                Term = datetime.timedelta(seconds=Second_Term)
                ddd = pre_time
                for n_diff in range(int(Diff)):    
                    ddd = ddd + Term
                    #print(str(ddd.year) + ',' + str(ddd.month) + ',' + str(ddd.day) + ',' + str(ddd.hour) + ',' + str(ddd.minute) + '\n')
                    f.write(str(ddd.year) + ',' + str(ddd.month) + ',' + str(ddd.day) + ',' + str(ddd.hour) + ',' + str(ddd.minute))
                    for wr in range(len_PRED+1):
                        f.write(',' + str(-99999))
                    f.write('\n')

    f.close()

In [5]:
num_epoch = 1200
num_unit = 240

#Make_LSTM_StepUp(42526, 8605, 8325, state_condition='Stateless', shuffle='True', peak_hour=1, pred_hour=24, LSTM_hour=24, 
#                 num_unit=num_unit, num_epoch=num_epoch, optimizer="Adam",loss_ftn='mean_squared_error')
#Make_LSTM_Onetime(42066, 8536, 8122, state_condition='Stateless', shuffle='True', peak_hour=1, pred_hour=24, LSTM_hour=24, 
#                 num_unit=num_unit, num_epoch=num_epoch, optimizer="Adam",loss_ftn='mean_squared_error')

#Make_LSTM_StepUp(42526, 8605, 8325, state_condition='Stateless', shuffle='False', peak_hour=1, pred_hour=24, LSTM_hour=24, 
#                 num_unit=num_unit, num_epoch=num_epoch, optimizer="Adam",loss_ftn='mean_squared_error')
#Make_LSTM_Onetime(42066, 8536, 8122, state_condition='Stateless', shuffle='False', peak_hour=1, pred_hour=24, LSTM_hour=24, 
#                 num_unit=num_unit, num_epoch=num_epoch, optimizer="Adam",loss_ftn='mean_squared_error')

In [6]:
num_epoch = 1200
num_unit = 40

#Make_LSTM_StepUp(42789, 8642, 8421, state_condition='Stateless', shuffle='True', peak_hour=1, pred_hour=24, LSTM_hour=36, 
#                 num_unit=num_unit, num_epoch=num_epoch, optimizer="Adam",loss_ftn='mean_squared_error')
#Make_LSTM_Onetime(42317, 8573, 8214, state_condition='Stateless', shuffle='True', peak_hour=1, pred_hour=24, LSTM_hour=36, 
#                 num_unit=num_unit, num_epoch=num|_epoch, optimizer="Adam",loss_ftn='mean_squared_error')

#Make_LSTM_StepUp(42789, 8642, 8421, state_condition='Stateless', shuffle='False', peak_hour=1, pred_hour=24, LSTM_hour=36, 
#                 num_unit=num_unit, num_epoch=num_epoch, optimizer="Adam",loss_ftn='mean_squared_error')
#Make_LSTM_Onetime(42317, 8573, 8214, state_condition='Stateless', shuffle='False', peak_hour=1, pred_hour=24, LSTM_hour=36, 
#                 num_unit=num_unit, num_epoch=num_epoch, optimizer="Adam",loss_ftn='mean_squared_error')

Make_LSTM_Onetime(42317, 8573, 8214, state_condition='Stateless', shuffle='False', peak_hour=1, pred_hour=24, LSTM_hour=36, 
                 num_unit=num_unit, num_epoch=num_epoch, optimizer="Adam",loss_ftn='mean_squared_error')

Onetime Output!
allDataSet:  (59104, 65)
allDataSet_size:  59104
allDataSet Type:  <class 'pandas.core.frame.DataFrame'>
allDataSet_size Type:  <class 'int'>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 40)                5040      
_________________________________________________________________
dense_1 (Dense)              (None, 24)                984       
Total params: 6,024
Trainable params: 6,024
Non-trainable params: 0
_________________________________________________________________


OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

In [ ]:
num_epoch = 1200
num_unit =30

Make_LSTM_Onetime(42317, 8573, 8214, state_condition='Stateless', shuffle='False', peak_hour=1, pred_hour=24, LSTM_hour=36, 
                 num_unit=num_unit, num_epoch=num_epoch, optimizer="Adam",loss_ftn='mean_squared_error')

In [ ]:
num_epoch = 1200
num_unit =50

Make_LSTM_Onetime(42317, 8573, 8214, state_condition='Stateless', shuffle='False', peak_hour=1, pred_hour=24, LSTM_hour=36, 
                 num_unit=num_unit, num_epoch=num_epoch, optimizer="Adam",loss_ftn='mean_squared_error')